Dr Oliviero Andreussi, olivieroandreuss@boisestate.edu
Notebook designed with the assistance of Christopher Orizaba

Boise State University, Department of Chemistry and Biochemistry

# Molecular Modeling Tools for the Computational Vibrations Lab {-}

Most chemistry applications of quantum mechanics (a.k.a. quantum chemistry) relies on a powerful commercial software called Gaussian. This code was first developed by a forefather of quantum chemistry and Nobel prize winner John Pople. However, Gaussian is a Fortran 77 code that requires an expensive license to run. For our applications we can achieve the same results using Python-based codes, at the expense of some computing time. In the following we will be using [PySCF](https://pyscf.org/index.html) for our quantum chemistry calculations, so we will need to install it on our Colab instance.

In [ ]:
import os, importlib, pkgutil

PROPERTIES_REPO = "/Users/olivieroandreussi/Dropbox/Courses/BSU_CHEM324_Material/Notebooks/PChemLab/properties"
EXT_PYSCF_DIR = os.path.join(PROPERTIES_REPO, "pyscf")          # .../properties/pyscf
EXT_PROP_DIR  = os.path.join(EXT_PYSCF_DIR, "prop")             # .../properties/pyscf/prop

# Set env var too (nice to have)
os.environ["PYSCF_EXT_PATH"] = PROPERTIES_REPO

# Now import core pyscf
import pyscf
import pyscf.prop

# Extend the search paths (this is the key)
if EXT_PYSCF_DIR not in pyscf.__path__:
    pyscf.__path__.append(EXT_PYSCF_DIR)

if EXT_PROP_DIR not in pyscf.prop.__path__:
    pyscf.prop.__path__.append(EXT_PROP_DIR)

importlib.invalidate_caches()

print("pyscf.__path__ =", list(pyscf.__path__))
print("pyscf.prop.__path__ =", list(pyscf.prop.__path__))
print("pyscf.prop submodules:", [m.name for m in pkgutil.iter_modules(pyscf.prop.__path__)])


In [ ]:
# @title PySCF Setup { display-mode: "form" }
# Import the main components of PySCF used in this worksheet
!pip install pyscf
!pip install pyberny
!pip install pyscf\[geomopt\]
!pip install git+https://github.com/pyscf/properties
from pyscf import gto, scf, dft, mp, cc
from pyscf.geomopt.berny_solver import optimize
from pyscf.hessian import rks, thermo 
from pyscf.prop.infrared import rks as rks_ir
from pyscf.prop.infrared import rhf as rhf_ir
#
from scipy.constants import physical_constants # we will need these for units conversion

Before we start, let us import the main modules that we will need for this lecture. 

In [ ]:
# @title Notebook Setup { display-mode: "form" }
# Import the main modules used in this worksheet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import time
# Load the google drive with your files
from google.colab import drive
drive.mount('/content/drive')
# The following needs to be the path of the folder with all your datafile in .csv format
base_path = '/content/drive/MyDrive/'

Set the local path, in case you want to save some of the results and plots from this notebook

In [ ]:
# @title Set Local Path { display-mode: "form" }
# The following needs to be the path of the folder with all your collected data in .csv format
local_path="Colab Notebooks/CompVib_Data/" # @param {type:"string"}
path = base_path+local_path

In [ ]:
# @title Utilities { display-mode: "form" }
def quadratic_vertex_fit(x, y):
    """Fit y = ax^2 + bx + c through 3 points; return vertex (x0, y0) and coefficients."""
    x = np.asarray(x)
    y = np.asarray(y)
    A = np.vstack([x**2, x, np.ones_like(x)]).T
    a, b, c = np.linalg.solve(A, y)
    x0 = -b / (2*a)
    y0 = a*x0**2 + b*x0 + c
    return x0, y0, a, b, c

def quadratic_fit_3pt(x, y):
    """Fit y = ax^2 + bx + c through 3 points and return a,b,c and vertex."""
    x = np.asarray(x)
    y = np.asarray(y)
    A = np.vstack([x**2, x, np.ones_like(x)]).T
    a, b, c = np.linalg.solve(A, y)
    x0 = -b / (2*a)
    y0 = a*x0**2 + b*x0 + c
    return a, b, c, x0, y0

def plot_pes_with_centered_quadratic(distances_A, energies_eh, show_fit=False, title="Diatomic PES"):
    x = np.asarray(distances_A)
    E = np.asarray(energies_eh)

    i0 = int(np.argmin(E))
    if i0 == 0 or i0 == len(E) - 1:
        raise ValueError("Minimum is at boundary; expand dmin/dmax or increase resolution.")

    # --- 3-point quadratic fit in standard form ---
    x3 = x[i0-1:i0+2]
    E3 = E[i0-1:i0+2]
    A = np.vstack([x3**2, x3, np.ones_like(x3)]).T
    a_std, b_std, c_std = np.linalg.solve(A, E3)

    # Convert to centered form: E = E0 + a (R - Req)^2
    Req = -b_std / (2*a_std)
    E0 = a_std*Req**2 + b_std*Req + c_std
    a = 2 * a_std  # curvature parameter in Eh / Å^2

    # Convert to relative energy (kJ/mol) for plotting
    Eh_to_kJmol = 2625.499638
    dE_kJmol = (E - E.min()) * Eh_to_kJmol

    # Local quadratic curve
    x_fit = np.linspace(x.min(), x.max(), 250)
    E_fit = E0 + 0.5*a*(x_fit - Req)**2
    dE_fit_kJmol = (E_fit - E.min()) * Eh_to_kJmol

    # ---- Plot ----
    plt.figure(figsize=(8.5, 5.2))
    plt.plot(x, dE_kJmol, marker='o', markersize=4, linewidth=1.8, label='PES')
    if show_fit:
        plt.plot(x_fit, dE_fit_kJmol, linewidth=2.2, label='Quadratic Fit')

    # Mark grid minimum
    plt.scatter([x[i0]], [(E[i0]-E.min())*Eh_to_kJmol], s=90, marker='x', color='red', label='Grid Minimum')

    # Mark quadratic minimum
    plt.axvline(Req, linestyle='--', linewidth=1.6)
    plt.scatter([Req], [(E0-E.min())*Eh_to_kJmol], s=70, marker='D', color='green', label='Quadratic Minimum')

    plt.xlabel("Bond distance $R$ (Å)")
    plt.ylabel(r"Relative energy $\Delta E$ (kJ/mol)")
    plt.title(title)
    plt.grid(True, alpha=0.25)
    plt.legend(loc='upper right')

    # Info box
    info = (
        f"Grid min R = {x[i0]:.6f} Å\n"
        f"Quad  Re  = {Req:.6f} Å\n\n"
        "Fit form:\n"
        "E(R) = E(Re) + 0.5 * a (R - Re)^2\n\n"
        f"a = {a:.6e} Eh/Å²"
    )

    plt.text(
        0.02, 0.98, info,
        transform=plt.gca().transAxes,
        verticalalignment='top',
        bbox=dict(facecolor='white', edgecolor='black')
    )

    plt.tight_layout()
    plt.show()

    print("Centered quadratic fit:")
    print("E(R) = E(Re) + 0.5 * a (R - Re)^2")
    print(f"Re = {Req:.10f} Å")
    print(f"a  = {a:.10e} Eh/Å^2")

    return a, Req, E0

def plot_hessian(mol,H):
    natm = H.shape[0]
    # Flatten (natm,3,natm,3) -> (3N, 3N)
    Hcart = np.transpose(H, (0, 2, 1, 3)).reshape(3*natm, 3*natm)
    Hsym = 0.5 * (Hcart + Hcart.T)
    Hsym[np.abs(Hsym) < 1e-8] = 0

    # Labels: O1x, O1y, O1z, H2x, ...
    axes = ("x", "y", "z")
    labels = [f"{mol.atom_symbol(i)}{i}{ax}"
          for i in range(natm) for ax in axes]

    fig, ax = plt.subplots(figsize=(6, 5))
    v = np.max(np.abs(Hsym))
    im = ax.imshow(Hsym, cmap="coolwarm", origin="lower", vmin=-v, vmax=v)
    fig.colorbar(im, ax=ax, label="H (Eh/Bohr² or Eh/(Bohr·rad))")

    ax.set_xticks(range(3*natm)); ax.set_xticklabels(labels, rotation=90)
    ax.set_yticks(range(3*natm)); ax.set_yticklabels(labels)

    ax.set_title("Molecular Hessian")
    ax.set_xlabel("Cartesian coordinate")
    ax.set_ylabel("Cartesian coordinate")
    plt.tight_layout()
    plt.show()

def plot_dynamical_matrix(mol,H,mass):
    #
    atom_coords = mol.atom_coords()
    mass_center = np.einsum('z,zx->x', mass, atom_coords) / mass.sum()
    atom_coords = atom_coords - mass_center
    natm = atom_coords.shape[0]
    D = np.einsum('pqxy,p,q->pqxy', H, mass**-.5, mass**-.5)
    #
    natm = D.shape[0]
    # Flatten (natm,3,natm,3) -> (3N, 3N)
    Dcart = np.transpose(D, (0, 2, 1, 3)).reshape(3*natm, 3*natm)
    Dsym = 0.5 * (Dcart + Dcart.T)
    Dsym[np.abs(Dsym) < 1e-8] = 0

    # Labels: O1x, O1y, O1z, H2x, ...
    axes = ("x", "y", "z")
    labels = [f"{mol.atom_symbol(i)}{i}{ax}"
          for i in range(natm) for ax in axes]

    fig, ax = plt.subplots(figsize=(6, 5))
    v = np.max(np.abs(Dsym))
    im = ax.imshow(Dsym, cmap="coolwarm", origin="lower", vmin=-v, vmax=v)
    fig.colorbar(im, ax=ax, label="D (Eh/Bohr² or Eh/(Bohr·rad))")

    ax.set_xticks(range(3*natm)); ax.set_xticklabels(labels, rotation=90)
    ax.set_yticks(range(3*natm)); ax.set_yticklabels(labels)

    ax.set_title("Molecular Dynamical Matrix")
    ax.set_xlabel("Cartesian coordinate")
    ax.set_ylabel("Cartesian coordinate")
    plt.tight_layout()
    plt.show()

def get_cartesian_mode(mol, analysis, masses, mode_index):
    """
    Returns Cartesian displacement vectors for a selected mode.
    mode_index: index in sorted freq list
    """
    modes = analysis["norm_mode"]      # shape (3N, 3N)
    # Select mode column
    q = modes[mode_index]
    # Divide by sqrt(mass)
    disp = q / np.sqrt(masses)[:, None]
    return disp

def make_vibration_frames(mol, disp, amplitude=0.5, nframes=60, coords_in_bohr=True, disp_in_bohr=True):
    bohr_to_ang = 0.529177210903
    coords = mol.atom_coords()
    if coords_in_bohr:
        coords = coords * bohr_to_ang  # now Å
    # Make sure disp is in Å too
    dispA = disp
    if disp_in_bohr:
        dispA = disp * bohr_to_ang
    atoms = [mol.atom_symbol(i) for i in range(mol.natm)]
    frames = []
    # endpoint=False avoids duplicating the first frame at 2π
    for t in np.linspace(0, 2*np.pi, nframes, endpoint=False):
        factor = amplitude * np.sin(t)
        new_coords = coords + factor * dispA
        xyz = [f"{mol.natm}", f"t={t:.6f}"]
        for sym, (x, y, z) in zip(atoms, new_coords):
            xyz.append(f"{sym} {x:.6f} {y:.6f} {z:.6f}")
        frames.append("\n".join(xyz))
    return "\n".join(frames) + "\n"

def show_vibration(frames):
    view = py3Dmol.view(width=500, height=400)
    view.addModelsAsFrames(frames, 'xyz')
    view.setStyle({'stick': {'radius': 0.20}, 'sphere': {'scale': 0.35}})
    view.zoomTo()
    view.animate({'loop': 'forward', 'interval': 30})  # smaller interval = smoother
    view.show()
    return view

def read_jdx(file, len_drop=2):
    drop_idxs = []
    with open(file) as f:
        for i, line in enumerate(f):
            if line.lstrip().startswith('#'):
                continue
            drop_idxs.append(i)
            if len(drop_idxs) == len_drop:
                break
    df = pd.read_csv(file, comment="#", sep=r"\s+", skiprows=drop_idxs, header=None, names=["wavenumber", "transmittance"], usecols=[0, 1])
    return df

## Visualize the Systems

The following module needs to be installed on Colab to visualize and generate the molecular systems that we will simulate. 

In [ ]:
# @title Install and load RDKit, CirPy, and Py3DMol { display-mode: "form" }
!pip install rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
!pip install cirpy
import cirpy
! pip install py3Dmol
import py3Dmol

In particular we can use them to draw the molecules in our experiments. While for some molecules you can just write their names and RDKit will plot them, for most molecules you will need to provide their SMILES or their CAS numbers.  Luckily, CIRpy can usually find SMILES for you, if you type the common name correctly or if you know the CAS number. 


In [ ]:
# @title Choose the molecule to draw { display-mode: "form" }
input = 'formic acid' # @param {type:"string"}
input_type = 'name' # @param ["smiles", "name", "cas"] {allow-input: true}
if input_type != 'smiles' :
    smiles=cirpy.resolve( input, 'smiles')
else:
    smiles=input
img = Draw.MolToImage( Chem.MolFromSmiles(smiles), size=(300, 300) )
display(img)

In [ ]:
xyz = cirpy.resolve(input,'xyz')  # for a pyscf.gto.Mole object
view = py3Dmol.view(width=400, height=300)
view.addModel(xyz, 'xyz')
view.setStyle({'stick': {'radius': 0.20}, 'sphere': {'scale': 0.35}})
view.zoomTo()
view.show()

## Potential Energy Surface of Diatomic Molecules

We can now create a molecule (aka `Mole`) object in PySCF, which holds static molecular data: atoms, basis, charges, integrals, symmetry. We need to manually assign the atoms (elements and coordinates) to it, and setup our choice of  basis set. We will then be able to use this molecule object to run calculations of its energy with different types of methods. The combination of method and basis set is usually referred to as *model chemistry*. Total energy calculations of given atomic coordinates are also called single-point energy calculations.  

In [ ]:
# @title Diatomic Molecule PES Scan { display-mode: "form" }
atom1 = 'H'            # @param {type:"string"}
atom2 = 'H'            # @param {type:"string"}  
method = 'DFT-B3LYP'          # @param ["HF", "DFT-B3LYP", "DFT-PBE", "MP2", "CCSD", "CCSD(T)"]
basis_set = "STO-3G"   # @param ["STO-3G", "4-31G", "6-31G*", "6-311G**", "cc-pVDZ", "cc-pVTZ"]
dmin = 0.2             # @param {type:"number"}  # bohr
dmax = 3.0             # @param {type:"number"}  # bohr
nsteps_energy = 40     # @param {type:"integer"} # samples for energy curves
show_fit = False          # @param {type:"boolean"} # whether to show the quadratic fit curve in the plot

# ---------- Build Molecule (bond along x) ----------
def make_mol(atom1, atom2, R, basis):
    atom = f"{atom1} {-R/2} 0 0; {atom2} {R/2} 0 0"
    mol = gto.Mole()
    mol.atom = atom
    mol.unit = 'Angstrom'
    mol.basis = basis
    mol.spin = 0
    mol.charge = 0
    mol.verbose = 0
    mol.build()
    return mol

distances = np.linspace(dmin, dmax, nsteps_energy)
PES = []
for R in distances:
    mol = make_mol(atom1, atom2, R, basis_set)
    if method == 'HF':
        mf = scf.RHF(mol)
        energy = mf.kernel()
    elif method == 'DFT-B3LYP':
        mf = dft.RKS(mol,xc='B3LYP')
        energy = mf.kernel()
    elif method == 'DFT-PBE':
        mf = dft.RKS(mol,xc='PBE')
        energy = mf.kernel()
    elif method == 'MP2':
        mf = scf.RHF(mol)
        energy = mf.kernel()
        mp2_energy = mp.MP2(mf).kernel()
        energy += mp2_energy
    elif method == 'CCSD':
        mf = scf.RHF(mol)
        energy = mf.kernel()
        ccsd_energy = cc.CCSD(mf).kernel()[0]
        energy += ccsd_energy
    elif method == 'CCSD(T)':
        mf = scf.RHF(mol)
        energy = mf.kernel()
        ccsd_t_energy = cc.CCSD(mf).kernel()[0] + cc.CCSD(mf).ccsd_t()[0]
        energy += ccsd_t_energy
    PES.append(energy)
PES = np.array(PES)

a, re_est, E_est = plot_pes_with_centered_quadratic(
    distances, PES, show_fit=show_fit,
    title=f"{atom1}{atom2} PES ({method}, {basis_set})"
)
print(f"Quadratic estimate: Re = {re_est:.8f} Angstrom, E = {E_est:.10f} Eh")

# Hessian, Dynamical Matrix, and Vibrational Frequencies

In [ ]:
# @title Compute the Molecular Hessian { display-mode: "form" }
input = 'formic acid' # @param {type:"string"}
basis_set = 'STO-3G'   # @param {type:"string"}
functional = 'B3LYP'  # @param {type:"string"}
#----------------------------------------------------------------
# STEP 1: Generate the system initial configuration
#----------------------------------------------------------------
# This converts the molecule name into an xyz file that is then used to extract the atomic coordinates
xyz = ''.join(string+'\n' for string in cirpy.resolve(input,'xyz').split('\n')[2:])
# This creates an empty PySCF molecule object
molecule = gto.Mole()
# This fills the molecule object with the atomic coordinates and specifies the units
molecule.atom = xyz
molecule.unit = 'Angstrom'
# This allows you to specify the basis set associated with the molecule
molecule.basis = basis_set
# This controls how much output is produced
molecule.verbose = 0
# This creates the molecule
molecule.build()
# 
#----------------------------------------------------------------
# STEP 2: Optimize the geometry of the molecule
#----------------------------------------------------------------
# Now that the molecule is build, we can setup a calculation object using this molecule
molecule_dft = dft.RKS(molecule,xc=functional) # THIS RUNS DFT, IF YOU WANT TO RUN HF YOU NEED TO CHANGE THIS!!!
# We can run the calculation
molecule_dft.run()
print(f"HF Energy of the initial structure:   {molecule_dft.e_tot} Ha")
# To run a geometry optimization we need to feed the calculation object to the optimize function
molecule_opt = optimize(molecule_dft) # The output is a new molecule object with the optimized geometry
# With the new optimized molecule we can setup a new calculation (in principle different from the previous one) 
# and recompute the energy 
molecule_opt_dft = dft.RKS(molecule_opt,xc=functional) # THIS RUNS DFT, IF YOU WANT TO RUN HF YOU NEED TO CHANGE THIS!!!
molecule_opt_dft.run()
print(f"HF Energy of the optimized structure: {molecule_opt_dft.e_tot} Ha")
#
#----------------------------------------------------------------
# STEP 3: Compute the Hessian matrix
#----------------------------------------------------------------
H = rks.Hessian(molecule_opt_dft).kernel()
plot_hessian(molecule_opt,H)

In [ ]:
# @title Adjust the Isotope Mass { display-mode: "form" }
atom_index = 3  # @param {type:"integer"}  # index of the atom to change the mass (starting from 0)
new_mass_value = 1.007  # @param {type:"number"}  # new mass value in atomic mass units (amu)
#----------------------------------------------------------------
# STEP 4: Adjust the masses of the atoms of the molecule
#----------------------------------------------------------------
mass = molecule_opt.atom_mass_list()
new_mass = mass.copy()
#new_mass[3] = 1.007  # NOTE: this is just an example on how to change the mass of a given atom
# Note: the following shows how the Hessian is transformed into the dynamical matrix
plot_dynamical_matrix(molecule_opt,H,new_mass)
#
#----------------------------------------------------------------
# STEP 5: Diagonalize the dynamical matrix to get the vibrational frequencies
#----------------------------------------------------------------
analysis = thermo.harmonic_analysis(molecule_opt, H, mass=new_mass)
freqs_cm1 = np.array(analysis["freq_wavenumber"])
print("Vibrational frequencies (cm^-1):")
print(freqs_cm1)
normal_modes = analysis["norm_mode"]  # shape (natm, 3, natm*3) -> (natm*3, natm*3) after flattening

In [ ]:
# @title Visualize the Vibrational Modes { display-mode: "form" }
mode_index = 20  # @param {type:"integer"}  # choose a vibrational mode
if mode_index < 0 or mode_index >= len(freqs_cm1):
    raise ValueError(f"Invalid mode index. Must be between 0 and {len(freqs_cm1)-1}.")
print(f"Vibrational frequency of mode {mode_index} (cm^-1): {freqs_cm1[mode_index]:.2f}")
disp = get_cartesian_mode(molecule_opt, analysis, mass, mode_index)
frames = make_vibration_frames(molecule_opt, disp)
view = show_vibration(frames)

In [ ]:
# @title Compute IR Intensities { display-mode: "form" }
#----------------------------------------------------------------
# STEP 6: Compute the IR intensities 
#----------------------------------------------------------------
ir_calculator = rks_ir.Infrared(molecule_opt_dft) # NOTE: this runs the IR calculation at the DFT level, if you want to run it at the HF level you need to change this to rhf_ir.Infrared(molecule_opt_hf)
ir_intensities = ir_calculator.run().ir_inten
print("IR intensities (km/mol):")
print(ir_intensities)


In [ ]:
# @title Plot IR Spectrum and Compare with the Experiment { display-mode: "form" }
freq_scale = 0.892 # @param {type:"number"}
width = 50 # @param {type:"number"}
intensity_scale = 40 # @param {type:"number"}
experimental_file = '64-18-6-IR.jdx' # @param {type:"string"}
#
# Plot the IR spectrum and compare with the experimental one (if available)
#
fig, ax1, ax2 = ir_calculator.plot_ir(w=width,scale=freq_scale) # w is the width of the peaks in cm^-1, scale is a factor to adjust the peak positions according to model chemistry (check on NIST database)
if experimental_file:
    experimental_data = read_jdx(path+experimental_file)
    ax2.plot(experimental_data["wavenumber"], (1-experimental_data["transmittance"])*intensity_scale, color="black", linewidth=1.5, label="Experimental")
    ax2.legend(loc='upper right')